In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys

sys.path.append('../src')
from clustering_algos import opt_kmeans,  backtrack
np.random.seed(seed = 0)

In [3]:
m = [0,3,6]
data = np.random.normal(loc = m, scale = [0.5]*3, size = [1000,3])
data = np.sort(data.reshape([-1]))
#data = np.random.uniform(0,10,300)
#data = np.sort(data.reshape([-1]))
#data = np.arange(300)
k=9

In [ ]:
#1d optimal kmeans no weights
D1,B1 = opt_kmeans(data, k)

#1d optimal kmeans with weights
weights = np.ones(shape = [len(data)])
#weights[0:200] = 1000000
#weights = weights/np.sum(weights)  
D,B = opt_kmeans_with_weights(data, k, weights = weights)


In [ ]:
kb = 6
ints1 = backtrack(B1, len(data), kb)
ints = backtrack(B, len(data), kb)


In [ ]:
ints == ints1

In [ ]:
ints

In [ ]:
fig, ax = plt.subplots(1,1)
ax = sns.distplot(data, bins = 100, kde = False,ax = ax)
for i in range(1,len(ints)):
    mu = np.sum(data[int(ints[i-1]):int(ints[i])]*weights[ints[i-1]:ints[i]])/sum(weights[ints[i-1]:ints[i]])
    #print(mu)
    ax.axvline(x = mu, color = 'r', linestyle = '--')

In [4]:
from numba import jit

@jit(nopython = True)
def opt_kmeans_fast(data, k):
    D = np.zeros((data.shape[0]+1,k+1), dtype = np.float64)
    B = np.zeros((data.shape[0]+1,k+1), dtype = np.int64)
    data = data.astype(np.float64)
    
    dij = 0
    mij = 0
    for i in range(1,D.shape[0]):
        D[i,0] = np.inf
        dij = dij+ (i-1)/i*(data[i-1]-mij)**2
        D[i,1] = dij
        mij = (data[i-1]+(i-1)*mij)/i
        
    for k2 in range(2,k+1):
        for i in range(k2, data.shape[0]+1):
            temp = np.inf
            tmp_indx = -1
            dji = 0
            mji = data[i-1]
            for j in range(i,k2-1,-1):
                #print(j)

                if i == j:
                    dji = 0
                    mji = data[i-1]
                else:
                    dji = dji + (i-j)/(i-j+1)*(data[j-1]-mji)**2
                    mji = (data[j-1]+(i-j)*mji)/(i-j+1)
                #if i <=3:
                    #print(i,j,dji,mji)
                if dji + D[j-1,k2-1] < temp:
                    temp_indx = j
                temp = np.minimum(temp, dji + D[j-1,k2-1])
            D[i,k2] = temp
            B[i,k2] = temp_indx
            
    return D,B

@jit(nopython = True)
def opt_kmeans_fast2(data, k):
    D = np.zeros((data.shape[0], k ), dtype = np.float64)
    B = np.zeros((data.shape[0], k ), dtype = np.int64)
    data = data.astype(np.float64)
    
    di1 = 0
    mi1 = data[0]
    for i in range(1, D.shape[0]):
        di1 = di1 + (i)/(i+1)*(data[i]-mi1)**2
        D[i,0] = di1
        mi1 = (data[i]+(i)*mi1)/(i+1)
        
    for k2 in range(1,k):
        for i in range(k2, data.shape[0]):
            temp = np.inf
            tmp_indx = -1
            dji = 0
            mji = data[i]
            for j in range(i,k2-1,-1):
                #print(j)

                if i == j:
                    dji = 0
                    mji = data[i]
                else:
                    dji = dji + (i-j)/(i-j+1)*(data[j]-mji)**2
                    mji = (data[j]+(i-j)*mji)/(i-j+1)
                #if i <=3:
                #print(i,j,dji + D[j-1,k2-1],temp, mji)
                if dji + D[j-1,k2-1] < temp:
                    temp_indx = j
                temp = np.minimum(temp, dji + D[j-1,k2-1])
            #print(temp, j, k2)
            D[i,k2] = temp
            B[i,k2] = temp_indx
            
    return D,B

def backtrack3(B, n, k):
    intervals = [n]
    nex = n
    for j in range(k-1, 0,-1):
        intervals.append(int(B[nex-1, j]))
        nex = intervals[-1]
        
    intervals.append(0)
    return intervals[::-1]

def backtrack2(B, n, k):
    intervals = [n]
    nex = n+1
    for j in range(k, 1,-1):
        intervals.append(int(B[nex-1, j]))
        nex = intervals[-1]
        
    intervals.append(0)
    return intervals[::-1]

In [5]:
%%time
#data = np.sort(np.array([4.0, 4.1, 4.2, -50, 200.2, 200.4, 200.9, 80, 100, 102]))
D1,B1 = opt_kmeans_fast2(data, 4)

CPU times: user 1.06 s, sys: 58.5 ms, total: 1.12 s
Wall time: 1.23 s


In [12]:
ints, means

([0, 999, 1521, 2006, 3000],
 [-0.0037063325103819755,
  2.59947612989535,
  3.4113191858705765,
  5.986772752319333])

In [6]:
ints = backtrack3(B1, len(data), 4)
means = [np.mean(data[ints[i]:ints[i+1]]) for i in range(len(ints)-1)]

In [9]:
%%time
D2,B2 = opt_kmeans(data, 4)

CPU times: user 199 ms, sys: 1.7 ms, total: 201 ms
Wall time: 199 ms


In [10]:
np.all(B2 == B1)

True